so the point of this one is to get a status check on our current follows, comparing them to last week's follows. If we find any follows that were previously there and are not we need to check to see if 1. that user exists and wasn't just banned/deleted, and 2. if we are following that user if they exist, and if 1 and 2 then unfollow them.

In [1]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.5 MB/s eta 0:00:00


In [2]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.1/351.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.4/682.4 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [3]:
from atproto import Client
from google.colab import userdata
# aws stuff
import boto3
from botocore.exceptions import ClientError
# json necessary to parse secret string, and write/read s3 objects
import json
# these imports are to use github apis to do logging, base64 is to parse the json
import requests
import base64
# datetime for logging
import datetime
import zoneinfo

In [4]:
AWS_KEY = userdata.get('aws_access_key')
AWS_SECRET_KEY = userdata.get('aws_secret_access_key')
REGION = userdata.get('aws_region')
SECRETS_ID = userdata.get('aws_secretsmanager_id')

# this file doesn't need ddb access
# DDB = 'dynamodb'
S3 = 's3'
# DDB_TABLE = 'rickybot-ddb'
S3_BUCKET = 'rickybot-s3'
S3_KEY_FOLLOWING_YOU = 'STATUS-FOLLOWING-YOU' # unlike the others that have a key determined by the day of the week, this will check the same spot every time.
S3_KEY_WHO_YOU_FOLLOW = 'STATUS-WHO-YOU-FOLLOW'


USER_TIMEZONE = "US/Eastern"
FILE_PATH = "LOGGING_STATUS_02.txt"
BRANCH = "main"

In [5]:
# connect to aws
try:
  aws_session = boto3.Session(
          aws_access_key_id = AWS_KEY,
          aws_secret_access_key = AWS_SECRET_KEY,
          region_name = REGION
      )
except:
  print('failed to begin AWS session')
  # return with error
  # this is the only error that we can't log to github, because we never got the credentials

In [6]:
# then connect to secrets manager
try:
  secrets_client = aws_session.client('secretsmanager')
  secret_value = secrets_client.get_secret_value(SecretId=SECRETS_ID)
  secret_string = secret_value['SecretString']
  secret_map = json.loads(secret_string)
except:
  print('failed to reach aws secrets manager')
  # return with error

In [7]:
# create constants from the values in the secrets manager
BSKY_USERNAME = secret_map['bsky_username']
BSKY_PASSWORD = secret_map['bsky_password']
GITHUB_TOKEN = secret_map['github_token']
GITHUB_REPO = secret_map['github_user/repo']
HUGGING_TOKEN = secret_map['hugging_token']

In [8]:
# before the program starts let's set up the logging function so we can insert it at any point where our program could break
def logging_status(logging_text):
  # LOGGING ALL THE CHANGES TO OUR LOGGING FILE IN GITHUB
  cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))
  datetime_object = datetime.datetime.fromtimestamp(cur_timestamp.timestamp())
  date_only = str(datetime_object.date())
  commit_message = "Logging status of followers on " + date_only


  # Step 1: Get the file's current content and SHA
  url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{FILE_PATH}"
  headers = {"Authorization": f"token {GITHUB_TOKEN}"}
  response = requests.get(url, headers=headers)
  response_json = response.json()

  # Decode the content of the file
  file_sha = response_json["sha"]
  content = base64.b64decode(response_json["content"]).decode("utf-8")

  # Step 2: Modify the file content
  new_content = content + date_only + ': ' + logging_text + '\n'
  encoded_content = base64.b64encode(new_content.encode("utf-8")).decode("utf-8")

  # Step 3: Push the updated content
  data = {
      "message": commit_message,
      "content": encoded_content,
      "sha": file_sha,
      "branch": BRANCH,
  }
  update_response = requests.put(url, headers=headers, json=data)

  if update_response.status_code == 200:
      print("Logging file updated successfully! Here's what was added to the logs:")
      print(date_only + ": " + logging_text)
  else:
      print(f"Error: {update_response.json()}")

In [9]:
# initialize the s3 client and get the bucket
try:
  s3 = aws_session.client(S3)
  buckets = s3.list_buckets()
  bucket = s3.list_objects_v2(Bucket=S3_BUCKET)
except:
  print('ERROR - failed to get s3 bucket')
  logging_status('ERROR - failed to get s3 bucket')
  # return with error

In [10]:
# log in to bluesky
try:
  client = Client()
  client.login(BSKY_USERNAME, BSKY_PASSWORD)
except:
  print('ERROR - failed to log in to bluesky')
  logging_status('ERROR - failed to log in to bluesky')

In [11]:
# get the current list of bluesky followers we'll create a map of follower_id: follow_uri, and follow_uri will identify if we follow them
try:
  current_followers = {}
  followers_count = client.get_profile(actor=BSKY_USERNAME).followers_count
  remaining = followers_count
  next_page = ''
  while remaining > 0:
    print (f'starting the next page of followers. {remaining} followers remaining to check.')
    cur_limit = min(remaining, 100)
    remaining -= cur_limit
    followers = client.get_followers(actor=BSKY_USERNAME, cursor= next_page, limit=cur_limit)
    next_page = followers.cursor
    followers = followers.followers
    for user in followers:
      did = user.did
      handle = user.handle
      follow_uri = user.viewer.following
      # print(handle, did, follow_uri)
      # handles can change we only want to deal with the did
      current_followers[handle] = follow_uri
except:
  print('ERROR - failed to gather followers')
  logging_status('ERROR - failed to gather followers')
  # return early


starting the next page of followers. 12077 followers remaining to check.
starting the next page of followers. 11977 followers remaining to check.
starting the next page of followers. 11877 followers remaining to check.
starting the next page of followers. 11777 followers remaining to check.
starting the next page of followers. 11677 followers remaining to check.
starting the next page of followers. 11577 followers remaining to check.
starting the next page of followers. 11477 followers remaining to check.
starting the next page of followers. 11377 followers remaining to check.
starting the next page of followers. 11277 followers remaining to check.
starting the next page of followers. 11177 followers remaining to check.
starting the next page of followers. 11077 followers remaining to check.
starting the next page of followers. 10977 followers remaining to check.
starting the next page of followers. 10877 followers remaining to check.
starting the next page of followers. 10777 follower

In [12]:
# now we get the old list of our followers from s3
# then if we find an old list in our bucket we compare it to the new list and unfollow anyone who is not in our new list.
count_removed = 0
count_failed_removal = 0
success_for_followers = False
try:
  s3.head_object(Bucket=S3_BUCKET, Key=S3_KEY_FOLLOWING_YOU)
  print("Object existed in s3 bucket.")
  response = s3.get_object(Bucket=S3_BUCKET, Key=S3_KEY_FOLLOWING_YOU)
  # creates a list from the json info in the s3 bucket
  old_followers = json.loads(response["Body"].read())
  print(old_followers, type(old_followers))
  for user_did, follow_uri in old_followers.items():
    if user_did not in current_followers:
      print('not in followers', user_did)
      try:
        client.delete_follow(follow_uri)
        count_removed += 1
        # print(cur)
      except Exception as e:
        # print(f'failed on {i} - uri: {follow_uri} \n {e}')
        count_failed_removal +=1
  follow_diff = len(current_followers) - len(old_followers)
  logging_status(f'followers status - {"up" if follow_diff >= 0 else "down"} {abs(follow_diff)} followers this week. {count_removed + count_failed_removal} users stopped following. {count_removed} were successfully unfollowed, with {count_failed_removal} failures.')
except s3.exceptions.ClientError as e:
  if e.response["Error"]["Code"] == "404": # object was not found at the key
    err = "ERROR - there was no previous followers list found in the s3 bucket. Adding the new list of followers."
    print(err)
    logging_status(err)
  else:
    err = "ERROR - failed to access the s3 bucket to get previous followers."
    print(err)
    logging_status(err)
finally:
  # regardless of if we were able to perform the comparison or not we want to input our new list into storage
  try:
    s3.put_object(
        Bucket=S3_BUCKET,
        Key=S3_KEY_FOLLOWING_YOU,
        Body=json.dumps(current_followers),
        ContentType="application/json"
    )
    success_for_followers = True
  except Exception as e:
      print(f"ERROR - failed to upload new followers map to s3: {e}")
      logging_status(f"ERROR - failed to upload new followers map to s3: {e}")


ERROR - there was no previous followers list to compaare to found in the s3 bucket. Adding the new list of followers.
Logging file updated successfully! Here's what was added to the logs:
2025-02-23: ERROR - there was no previous followers list to compare to found in the s3 bucket. Adding the new list of followers.


In [13]:
# now we're going to repeat this process with our account follows
# generate our account's follows list
cur_who_you_follow = {}
following = client.get_profile(actor=BSKY_USERNAME).follows_count
print(f'currently following {following} users')
next_page = ''
remaining = following
while remaining > 0:
  print (f'starting the next page of follows. {remaining} follows remaining to check.')
  cur_limit = min(remaining, 100)
  remaining -= cur_limit
  follows = client.get_follows(actor=BSKY_USERNAME, cursor= next_page, limit=cur_limit)
  next_page = follows.cursor
  follows = follows.follows
  for follow in follows:
    did = follow.did
    # handle = follow.handle
    # following is whether they are following you, it looks like followed_by is if they are following you back.
    # follows_you = 'FALSE' if follow.viewer.followed_by == None else 'TRUE' # we actually don't need this value at all, but we need a value in the map so might as well put something hypothetically useful and small in memory
    follow_uri = follow.viewer.following
    cur_who_you_follow[did] = follow_uri

currently following 27834 users
starting the next page of follows. 27834 follows remaining to check.
starting the next page of follows. 27734 follows remaining to check.
starting the next page of follows. 27634 follows remaining to check.
starting the next page of follows. 27534 follows remaining to check.
starting the next page of follows. 27434 follows remaining to check.
starting the next page of follows. 27334 follows remaining to check.
starting the next page of follows. 27234 follows remaining to check.
starting the next page of follows. 27134 follows remaining to check.
starting the next page of follows. 27034 follows remaining to check.
starting the next page of follows. 26934 follows remaining to check.
starting the next page of follows. 26834 follows remaining to check.
starting the next page of follows. 26734 follows remaining to check.
starting the next page of follows. 26634 follows remaining to check.
starting the next page of follows. 26534 follows remaining to check.
st

In [14]:
# with a hashmap of our follows to iterate through we're going to do something very similar to the followers
# but this time we're going to see if they're in the previous week's follows, and see if they're in the followers hashmap
# and if we were following them last week but they're still not a follower, then we delete them.
# now we get the old list of our followers from s3
count_removed = 0
count_failed_removal = 0
success_for_who_we_follow = False
users_deleted = []
try:
  s3.head_object(Bucket=S3_BUCKET, Key=S3_KEY_WHO_YOU_FOLLOW)
  print("Object existed in s3 bucket.")
  response = s3.get_object(Bucket=S3_BUCKET, Key=S3_KEY_WHO_YOU_FOLLOW)
  # creates a list from the json info in the s3 bucket
  prev_who_we_were_following = json.loads(response["Body"].read())
  print(prev_who_we_were_following, type(prev_who_we_were_following))
  for user_did, follow_uri in cur_who_you_follow.items():
    # as we iterate what we want to find are people who Were in the previous list and are Not now in the follows list
    if user_did in prev_who_we_were_following and user_did not in current_followers:
      print(f'this user is an old follow but is still not a follower: {user_did}')
      try:
        client.delete_follow(follow_uri)
        count_removed += 1
        # print(cur)
        # if we're successfully able to delete the follow we shouldn't keep it in our list of follows too.
        users_deleted.append(user_did)
      except Exception as e:
        # print(f'failed on {i} - uri: {follow_uri} \n {e}')
        count_failed_removal +=1
  # now that we're done iterating through the dict we can safely remove all the users that we deleted and should not be included in it
  for user in users_deleted:
    del cur_who_you_follow[user]
  follow_diff = len(cur_who_you_follow) - len(prev_who_we_were_following)
  logging_status(f'who you follow status - {"up" if follow_diff >= 0 else "down"} {abs(follow_diff)} follows this week. {count_removed + count_failed_removal} users have aged out and were necessary to prune. {count_removed} were successfully unfollowed, with {count_failed_removal} failures.')
except s3.exceptions.ClientError as e:
  if e.response["Error"]["Code"] == "404": # object was not found at the key
    err = "ERROR - there was no previous list of who we follow found in the s3 bucket. Adding the new list of follows."
    print(err)
    logging_status(err)
  else:
    err = "ERROR - failed to access the s3 bucket to get previous list of who we follow."
    print(err)
    logging_status(err)
finally:
  # regardless of if we were able to perform the comparison or not we want to input our new list into storage
  try:
    s3.put_object(
        Bucket=S3_BUCKET,
        Key=S3_KEY_WHO_YOU_FOLLOW,
        Body=json.dumps(cur_who_you_follow),
        ContentType="application/json"
    )
    success_for_who_we_follow = True
  except Exception as e:
    err = f"ERROR - failed to upload new map of who we follow to s3: {e}"
    print(err)
    logging_status(err)


ERROR - there was no previous list of who we follow found in the s3 bucket. Adding the new list of follows.
Logging file updated successfully! Here's what was added to the logs:
2025-02-23: ERROR - there was no previous list of who we follow found in the s3 bucket. Adding the new list of follows.
Logging file updated successfully! Here's what was added to the logs:
2025-02-23: successfully uploaded current users that we follow as map to s3.


In [15]:
logging_status(f"s3 update - uploaded followers: {'SUCCESS' if success_for_followers else 'FAILURE'} | uploaded who we follow: {'SUCCESS' if success_for_who_we_follow else 'FAILURE'} ")

Logging file updated successfully! Here's what was added to the logs:
2025-02-23: s3 update - uploaded current followers: SUCCESS | uploaded who we follow: SUCCESS 
